In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/shelter-animal-outcomes/train.csv.gz')
train

In [ ]:
test = pd.read_csv('/kaggle/input/shelter-animal-outcomes/test.csv.gz')
test

In [ ]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')

In [ ]:
alldata = pd.concat([train,test], ignore_index = True)
alldata

In [ ]:
alldata['Breed'].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()

breed_text = tf.fit_transform(alldata['Breed'])

breed_text

In [ ]:
breed_text[0]

In [ ]:
pd.options.display.max_columns = 999

pd.DataFrame(breed_text.todense())

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components = 5)

breed_svd = svd.fit_transform(breed_text)

breed_svd

In [ ]:
breed_svd.shape

In [ ]:
alldata = pd.concat([alldata, pd.DataFrame(breed_svd)], axis = 1)

In [ ]:
alldata

In [ ]:
alldata['AgeuponOutcome'].unique()

In [ ]:
a = alldata[alldata['AgeuponOutcome'] == '0 years']
a

In [ ]:
def Age(x):
    if pd.isnull(x):
        return -1
    num = int(x.split(' ')[0])
    
    if 'year' in x:
        return num * 365
    elif 'month' in x:
        return num *30
    elif 'week' in x:
        return num *7
    else:
        return num
    
alldata['AgeuponOutcome'] = alldata['AgeuponOutcome'].apply(Age)
alldata

In [ ]:
a = alldata[alldata['AnimalID'] == 'A677462']
a

In [ ]:
alldata['DateTime'] = pd.to_datetime(alldata['DateTime'])

alldata['Years'] = alldata['DateTime'].dt.year
alldata['Months'] = alldata['DateTime'].dt.month
alldata['Days'] = alldata['DateTime'].dt.day
alldata['Hour'] = alldata['DateTime'].dt.hour
alldata['WeekDay'] = alldata['DateTime'].dt.weekday
alldata['Min'] = alldata['DateTime'].dt.minute
alldata['Week'] = alldata['DateTime'].dt.week

alldata

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure (figsize = (16,10))

sns.countplot(alldata['Hour'], hue = alldata['OutcomeType'])

In [ ]:
alldata['petname'] = alldata['Name'].notnull()
alldata

In [ ]:
plt.figure(figsize = (16,10))
sns.boxplot(alldata['OutcomeType'], alldata['AgeuponOutcome'], showfliers = False, hue = alldata['AnimalType'])

In [ ]:
plt.figure(figsize = (16,10))
sns.boxplot(alldata['OutcomeType'], alldata['AgeuponOutcome'], showfliers = False, hue = alldata['petname'])

In [ ]:
alldata2 = alldata.drop(columns = ['OutcomeType', 'OutcomeSubtype', 'ID', 'AnimalID', 'DateTime', 'Name'])
alldata2

In [ ]:
c = alldata2.columns[alldata2.dtypes ==object]
c

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in c:
    
    alldata2[i] = le.fit_transform(list(alldata2[i]))

In [ ]:
alldata2

In [ ]:
alldata2 = alldata2.fillna(-1)

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
v = np.where(train2.dtypes != np.float)[0]
v

In [ ]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(verbose = 100)

cbc.fit(np.array(train2), train['OutcomeType'], cat_features = np.where(train2.dtypes != np.float)[0])

result = cbc.predict_proba(np.array(test2))

In [ ]:
result

In [ ]:
result[0]

In [ ]:
sub

In [ ]:
sub.iloc[:,1:] = result
sub

In [ ]:
sub.to_csv('sub1.csv', index = 0)

In [ ]:
pd.Series(cbc.feature_importances_, index = train2.columns).sort_values()